In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torch.multiprocessing as _mp

import resnet_cifar10
from networks import NetNAPER, NetTMR, NetDMR, NetDRO
from utils import *

from tqdm import tqdm
import copy
import time
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import StrMethodFormatter

import sysconfig
print(sysconfig.get_paths()['include'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Additional Info when using cuda
if device.type == 'cuda':
    print('Using device :', device)
    print('GPU          :', torch.cuda.get_device_name(0))
else:
    import cpuinfo
    cpuinfo.get_cpu_info()['brand']

print("Pytorch version: ",torch.__version__)


In [ ]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

test_set = datasets.CIFAR10("cifar10", train=False, download=True, transform=test_transform)

testloaders = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=True)

# Config

In [3]:
run_time = 10

model_types = [20,32,44,56]
idss =        [[0, 1], [0,1],[0,1],[0,1]]

# ResNets

In [ ]:
base_times = []

# base
for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    evaluate_acc(models[0], testloaders, single=True, time_profiler=True)
    model_time = []
    for i in range(run_time):
        model_time.append(evaluate_acc(models[0], testloaders, single=True, time_profiler=True)[-1])
    base_times.append(model_time)

In [ ]:
meantime = 0.
for _ in range(10):
    sample_input= next(iter(testloaders))[0][0].unsqueeze(0).to(device)
    if device == "cuda": 
        torch.cuda.synchronize()
    # with torch.no_grad():
    startdmr = time.perf_counter()
    models[0](sample_input)
    if device == "cuda": 
        torch.cuda.synchronize()
    enddmr = time.perf_counter()
    meantime += (enddmr-startdmr)*1000
meantime /= 10
print("mean time: ", meantime)

# DRO

In [ ]:
DRO_times = []
DRO_times2 = []


for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelDRO = NetDRO(models[0])
    modelDRO.to(device)
    modelDRO.eval()
    print("model DRO ready")
    print("accuracy", evaluate_acc(modelDRO, testloaders))

    evaluate_acc(modelDRO, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelDRO.protected=True
    
    torch.cuda.synchronize() # data masuk
    for i in range(run_time):
        model_time.append(evaluate_acc(modelDRO, testloaders, single=True, time_profiler=True)[-1])
    DRO_times.append(model_time)

# save time data
with open("tmp_p1_mobile_dro.pkl", "wb") as f:
    pickle.dump(DRO_times, f)

# TMR

In [ ]:
TMR_times = []
TMR_times2 = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id

    model_name ="resnet"+str(mod)
    # model_name ="mobilenet"+str(mod)
    # mod = "mobilenet"

    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelTMR = NetTMR(models[0])
    modelTMR.to(device)
    modelTMR.eval()
    print("model TMR ready")
    print("accuracy", evaluate_acc(modelTMR, testloaders))

    evaluate_acc(modelTMR, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelTMR.protected=True
    for i in range(run_time):
        model_time.append(evaluate_acc(modelTMR, testloaders, single=True, time_profiler=True)[-1])
    TMR_times.append(model_time)
print(TMR_times)

# save time data
with open("tmp_p1_mobile_tmr.pkl", "wb") as f:
    pickle.dump(TMR_times, f)

# CBR / DMR

In [ ]:
DMR_times = []
DMR_times2 = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id
   
    model_name ="resnet"+str(mod)

    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    modelDMR = NetDMR(models[0], "", sum_model(models[0]))
    modelDMR.to(device)
    modelDMR.eval()
    print("model DMR ready")
    print("accuracy", evaluate_acc(modelDMR, testloaders))

    evaluate_acc(modelDMR, testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    modelDMR.protected=True
    modelDMR.debug = False
    for i in range(run_time):
        model_time.append(evaluate_acc(modelDMR, testloaders, single=True, time_profiler=True)[-1])
    DMR_times.append(model_time)

print(DMR_times)
print("Average of Total Inference Time (no fault)    :", "{:.4f}".format(np.mean(DMR_times[0])), "ms")

# save time data
with open("tmp_p1_mobile_dmr.pkl", "wb") as f:
    pickle.dump(DMR_times, f)

del modelDMR, models

# NAPER

In [ ]:
NaPER_times = []
NaPER_single = []
NaPER_times2 = []

# NAPER ENSEMBLE
for id, mod in zip(idss, model_types):
    models = []
    ids = id
    
    model_name ="resnet"+str(mod)
    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print("model 0 accuracy", evaluate_acc(models[0], testloaders))

    deltas, summs = get_sum_delta(models)
    modelDelta = NetNAPER(models, deltas, summs)
    modelDelta.protected=False
    print("model NaPER ready")
    print("ensemble accuracy",ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders))
    print("ensemble 0 accuracy",ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, limit_model=1))

    print("finish warming up..")
    model_time = []
    modelDelta.protected=True
    for i in range(run_time):
        model_time.append(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, single=True, time_profiler=True)[-1])
    NaPER_times.append(model_time)

    model_time = []
    for i in range(run_time):
        model_time.append(ens_evaluate_acc(wrappermodel=modelDelta, testloaders=testloaders, single=True,
                                           time_profiler=True, limit_model=1)[-1])
    NaPER_single.append(model_time)
print(NaPER_times)
with open("tmp_p1_mobile_naper.pkl", "wb") as f:
    pickle.dump(NaPER_times, f)

# Ensemble ResNet

In [ ]:
EnsRes_time = []

for id, mod in zip(idss, model_types):
    models = []
    ids = id
    model_name ="resnet"+str(mod)
    models_path = "models/"

    # load all models
    models = eval_model_load(str(mod), ids, model_name, models_path, compile=False)
    print(evaluate_acc(models[0], testloaders))

    print("model ensemble ready")
    print("accuracy",ens_evaluate_acc(models=models, testloaders=testloaders))
    ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)
    ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)
    print("finish warming up..")
    model_time = []
    for i in range(run_time):
        model_time.append(ens_evaluate_acc(models=models, testloaders=testloaders, single=True, time_profiler=True)[-1])
    EnsRes_time.append(model_time)
print(EnsRes_time)

# PLOT - DATA

In [13]:
# ACC score are hardcoded for faster visualization, but the model is available in the /models folder

        #  20      32      44      56
accs_10=[[91.72, 92.31,	92.92,	93.25], # resnet accuracy single
        [93.26,	93.82,	94.02,	94.65]] # resnet ensemble accuracy

        #   20      32      44      56
accs_100=[[66.92, 68.82, 70.04,	70.87], # resnet accuracy single
         [71.75, 73.61,	74.8,	74.82]] # resnet ensemble accuracy

accs_traffic =[[93.00, 93.50, 93.63, 94.74],
               [94.89, 95.04, 95.78, 96.02]]

acc_eft = [93.9, 74.37, 95.29]

cbcol = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']

# PLOT - Overhead No Fault 

In [ ]:
for i in range(4):
    p_base = np.array(base_times[i])
    p_tmr =  np.array(TMR_times[i])
    tmr_median = np.median(p_tmr)
    if i==0:
        p_tmr_2 = np.array(TMR_times[i]) / tmr_median
        p_naper = np.array(NaPER_times[i])[:,-1] / tmr_median
        p_dro =  np.array(DRO_times[i]) / tmr_median
        p_cbr =  np.array(DMR_times[i]) / tmr_median
    else:
        p_tmr_2 *= np.array(TMR_times[i]) / tmr_median
        p_naper *= np.array(NaPER_times[i])[:,-1] / tmr_median
        p_dro *=  np.array(DRO_times[i]) / tmr_median
        p_cbr *=  np.array(DMR_times[i]) / tmr_median
p_tmr_2 = np.power(p_tmr_2, 1/4)
p_naper = np.power(p_naper, 1/4)
p_dro = np.power(p_dro, 1/4)
p_cbr = np.power(p_cbr, 1/4)

In [ ]:
model = ['NAPER',
        #  'ResNet20+NaPER1',
         'CBR',
         'DRO',
         'TMR',]
model.reverse()

data_all = np.flip(np.array([
    p_naper,
    p_cbr,
    p_dro,
    p_tmr_2
]))

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5,2), dpi=200)

flierprops = dict(marker='.', markerfacecolor='black', markersize=2, linestyle='none')
medianprops = dict(linestyle='--', linewidth=1, color='black')
ax.boxplot(data_all.T, vert=True, medianprops=medianprops, flierprops=flierprops)

ax.set_xticklabels(model, fontsize=9)
ax.tick_params(axis='y', which='major', labelsize=9)
ax.set_ylabel('Normalized Inference Time\n in Normal Condition', fontsize=8.5)
ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}x"))
ax.yaxis.grid(True)

plt.ylim(ymax=1.03)
plt.show()


# PLOT - Convex Hull

In [ ]:
fig = plt.figure(figsize=(5,2), dpi=200)
plt.xlabel("Deadline (ms)", fontsize=9)
plt.ylabel("Accuracy (%)", fontsize=9)
plt.xlim(xmax=40)
plt.xticks(fontsize=9) 
plt.yticks(fontsize=9)


accs = accs_10

xticks = ["ResNet20"]
def get_mam(arr, naper1=False, naper2=False):    
    _avg = np.mean(np.array(arr),axis=1)
    _min =  np.min(np.array(arr),axis=1)
    _max =  np.max(np.array(arr),axis=1)
    return(_avg, _min, _max)




man_startpoint = 0
a,mi,ma = get_mam(base_times)
ta1,tmi1,tma1 = get_mam(np.array(EnsRes_time)[:,:,-1])
ta = np.concatenate([a[:-1], ta1[man_startpoint:]])
tmi = np.concatenate([mi, tmi1[man_startpoint:]])
tma = np.concatenate([ma, tma1[man_startpoint:]])
accs_here = np.concatenate([accs[0][:-1], accs[1]])
plt.step(np.concatenate([ta, [52]]), np.concatenate([accs_here, [accs_here[-1]]]), color=cbcol[4], label="Unprotected ResNet",
         linewidth=1, marker="P", markersize=4, linestyle="-", where="post")


single_endpoint = 1
man_startpoint = -4
ta,tmi,tma = get_mam(np.array(NaPER_single)[:,:, 0])
ta1,tmi1,tma1 = get_mam(np.array(NaPER_times)[:, :, -1])
ta = np.concatenate([ta[:single_endpoint], ta1[man_startpoint:]])
tmi = np.concatenate([tmi[:single_endpoint], tmi1[man_startpoint:]])
tma = np.concatenate([tma[:single_endpoint], tma1[man_startpoint:]])
accs_here = np.concatenate([accs[0][:single_endpoint], accs[1][man_startpoint:]])
plt.step(np.concatenate([ta,[50]]), np.concatenate([accs_here,[accs_here[-1]]]),  color=cbcol[0], label="NAPER", linewidth=1, marker="o", markersize=2, where="post")

ta,tmi,tma = get_mam(DMR_times)
plt.step(np.concatenate([ta,[52]]), np.concatenate([accs[0], [accs[0][-1]]]), color=cbcol[1], label="CBR", linewidth=1, marker="D", markersize=4, where="post")


ta,tmi,tma = get_mam(DRO_times)
plt.step(np.concatenate([ta,[50]]), np.concatenate([accs[0],[accs[0][-1]]]),  color=cbcol[2], label="TMR-MEM", linewidth=1, marker="o", markersize=2, where="post")


ta,tmi,tma = get_mam(TMR_times)
plt.step(np.concatenate([ta,[52]]), np.concatenate([accs[0], [accs[0][-1]]]), color=cbcol[3], label="TMR", linewidth=1, marker="o", markersize=4, where="post")


plt.legend(bbox_to_anchor=(0.5, -0.3), ncol=5, loc='upper center', fontsize=9)

# PLOT - CIFAR10/CIFAR100 Accuracy

In [ ]:
# Adjust figure size to accommodate right-side legend
fig, ax = plt.subplots(1, 3, figsize=(7,2.5), dpi=200)
fig.subplots_adjust(wspace=0.2)  # Make room for legend on right

t = 0

### CIFAR100
for i in range(3):
    ax1 = ax[i]
    accs = [accs_10, accs_100, accs_traffic]

    # Define the x-values
    x = [20, 30, 40, 50] # A6000
    # x = [200, 300, 400, 500] # Jetson

    # Define the y-values for each category
    tmr = [accs[i][0][0], accs[i][0][1], accs[i][0][2], accs[i][0][3]]
    cbr = [accs[i][0][1], accs[i][0][2], accs[i][0][3], accs[i][0][3]]
    tmrm = [accs[i][0][0], accs[i][0][2], accs[i][0][3], accs[i][0][3]]
    unprotected_resnet = [accs[i][1][-1]]*len(x)
    naper = [accs[i][1][1],accs[i][1][2],accs[i][1][-1],accs[i][1][-1]]
    eft = [acc_eft[i]] * len(x)

    # Create the line plots with different markers
    ax1.plot(x, unprotected_resnet, marker='x', label='Unprot.\nResNet', color='black', linestyle='-', markersize=4)
    ax1.plot(x, tmr, marker='o', label='TMR', color=cbcol[1], linestyle='-', markersize=4)
    ax1.plot(x, cbr, marker='D', label='CBR', color=cbcol[2], linestyle='-', markersize=4)
    ax1.plot(x, tmrm, marker='*', label='DRO', color=cbcol[3], linestyle='-', markersize=4)
    ax1.plot(x, eft, marker='+', label='EFT', color=cbcol[4], linestyle='-', markersize=6)
    ax1.plot(x, naper, marker='^', label='NAPER', color=cbcol[0], linestyle='-', markersize=4)

    # Customize the plot
    ax1.set_xticks(x)
    ax1.grid(True, linestyle='--', alpha=0.3)
    ax1.set_axisbelow(True)
    ax1.set_xlabel('Latency Deadline (ms)', fontsize=9)

    # Set tick parameters
    ax1.tick_params(axis='both', which='major', labelsize=8)  # Smaller tick labels


# Set individual y-labels
ax[0].set_ylabel('Accuracy. (%)', fontsize=9)

# Set y-axis limits
ax[0].set_ylim(ymin=91, ymax=95.8)
ax[1].set_ylim(ymin=66, ymax=76.2)
ax[2].set_ylim(ymin=92, ymax=96.8)

ax[0].set_yticks([91, 93, 95])
ax[1].set_yticks([66, 70, 75])
ax[2].set_yticks([92, 95, 96])

# Add dataset titles inside each subplot
ax[0].set_title('CIFAR-10', fontsize=8, y=0.9)
ax[1].set_title('CIFAR-100', fontsize=8, y=0.9)
ax[2].set_title('GTSRB', fontsize=8, y=0.9)

# Move legend to bottom
plt.legend(bbox_to_anchor=(0.5, -0.15), 
          loc='center',
          ncol=6,  # Three columns
          fontsize=8,
          bbox_transform=plt.gcf().transFigure)  # Use figure coordinates

# plt.tight_layout()
plt.show()